# **Creacion de Datos**
### *Autor: Mario Daniel Ortega Garcia*


## Generación de Datos Simulados

Para la creación de datos se generó un conjunto de aproximadamente **100,000 registros de ventas** simuladas de model kits de **_Mobile Suit Gundam: Iron-Blooded Orphans_**, abreviado a partir de ahora como **IBO**.

Estas ventas ficticias se distribuyen entre los años **2022, 2023 y 2024**, y fueron creadas con un script en **Python** que toma en cuenta:

- Una lista de productos (Gunplas de IBO)
- Una lista de países con su continente respectivo
- Pesos asignados a cada continente para reflejar distribución desigual de ventas
- Distribución no uniforme en las unidades vendidas
- Fechas aleatorias dentro del rango mencionado

---

### Script utilizado para la creación de los datos


In [ ]:

import pandas as pd
import random
from datetime import datetime, timedelta

# Cargar archivos
productos = pd.read_csv("IBO_Gunplas.csv")
paises = pd.read_csv("Paises con continene.csv")

# Obtener listas
lista_productos = productos['id_nuevo'].dropna().unique().tolist()
paises = paises.dropna(subset=["id_pais", "continente"])
lista_paises = paises[['id_pais', 'continente']].copy()

# Pesos por continente
pesos_continente = {
    "Asia": 0.4,
    "América": 0.25,
    "Europa": 0.2,
    "África": 0.1,
    "Caribe": 0.05
}
lista_paises.loc[:, 'peso'] = lista_paises['continente'].map(pesos_continente)
lista_paises = lista_paises.dropna(subset=['peso']).reset_index(drop=True)

# Función para generar fechas aleatorias
def fecha_aleatoria():
    inicio = datetime(2022, 1, 1)
    fin = datetime(2024, 12, 31)
    return inicio + timedelta(days=random.randint(0, (fin - inicio).days))

# Generar ventas simuladas
num_ventas = 100_000
ventas_simuladas = []

for i in range(1, num_ventas + 1):
    fecha = fecha_aleatoria().strftime('%Y-%m-%d')
    id_producto = random.choice(lista_productos)
    pais = lista_paises.sample(weights=lista_paises['peso']).iloc[0]['id_pais']

    opciones = list(range(1, 21))
    pesos = [0.13]*5 + [0.02]*7 + [0.08]*5 + [0.01]*3
    suma_pesos = sum(pesos)
    pesos = [p/suma_pesos for p in pesos]
    unidades = random.choices(opciones, weights=pesos, k=1)[0]
    
    ventas_simuladas.append([i, fecha, id_producto, pais, unidades])

# Guardar como CSV
df_ventas = pd.DataFrame(ventas_simuladas, columns=['id_venta', 'fecha_venta', 'id_producto', 'id_pais', 'unidades'])
df_ventas.to_csv("Ventas_gunplas_simuladas.csv", index=False)
print("Archivo guardado como Ventas_gunplas_simuladas.csv")


### **UTILIZACION DE HERRAMIENTA LLM**:

Parte del código fue desarrollado con una herramenita LLM, sin embargo se ajustado manualmente para adaptarse a los objetivos del ejercicio.
Mas adelante se comentará de otro scripts hechos para desarrollar un Dataset mas "Realista"


##  Ensuciado de Datos Simulado

Para practicar técnicas de limpieza y validación de datos, se generó un conjunto de datos **ensuciados de forma intencional y aleatoria** a partir del dataset limpio de ventas simuladas.

Este proceso de ensuciado incluye errores comunes que se encuentran en datos reales, tales como:

- Fechas con formatos incorrectos o valores no válidos  
- Identificadores (`id_producto`, `id_pais`) con valores inválidos o fuera de rango  
- Valores numéricos expresados en texto (por ejemplo, `"muchas"`, `"??"`)  
- Presencia de valores nulos distribuidos aleatoriamente  
- Filas duplicadas para simular registros repetidos o erróneos  

Este dataset ensuciado es ideal para poner en práctica procesos de **limpieza de datos, manejo de errores y validación**, preparando los datos para un análisis confiable y efectivo.

### Script utilizado para el Ensuciado de los datos


In [4]:

import pandas as pd
import numpy as np


# Cargar el archivo limpio
df = pd.read_csv("Ventas_gunplas_simuladas.csv")

# Introducir valores nulos aleatorios
for col in ["id_venta", "Fecha_venta", "id_producto", "Id_Pais", "unidades"]:
    idx = df.sample(frac=0.01).index  # 1% de nulos por columna
    df.loc[idx, col] = np.nan

# Modificar fechas a un formato malo o texto
malas_fechas = df.sample(frac=0.005).index
df.loc[malas_fechas, "Fecha_venta"] = "fecha desconocida"

# Introducir IDs inválidos
df.loc[df.sample(frac=0.005).index, "id_producto"] = 99999  # ID que no existe
df.loc[df.sample(frac=0.005).index, "Id_Pais"] = -1  # ID de país inválido

# Unidades como texto
df.loc[df.sample(frac=0.005).index, "unidades"] = "muchas"
df.loc[df.sample(frac=0.005).index, "unidades"] = "??"

# Duplicar algunas filas
df = pd.concat([df, df.sample(frac=0.01)], ignore_index=True)

# Guardar archivo sucio
df.to_csv("Ventas_gunplas_sucias.csv", index=False)
print("Archivo sucio generado.")


FileNotFoundError: [Errno 2] No such file or directory: 'Ventas_gunplas_simuladas.csv'

### **Siguiente paso**
Con este archivo listo para limpiear pasaremos al siguiente punto que es "Limpiado del dataset"

Para acceder a el ve a la carpeta "2. Limpieza con Pandas"